In [ ]:
df1 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="1 bedroom flat")
df2 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="2 bedroom flat")
df3 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="3 bedroom flat")
df4 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="2 bedroom house")
df5 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="3 bedroom house")
df6 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="4 bedroom house")

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import pandas as pd
import pmdarima as pm
import matplotlib.pyplot as plt

In [16]:
import pandas as pd

file_path = "../../data/raw/moving-annual-rent-2025.xlsx"
xls = pd.ExcelFile(file_path)

# Get all sheet names except "All properties"
sheets = [s for s in xls.sheet_names if s != "All properties"]
print("Sheets included:", sheets)

all_data = []

for sheet in sheets:
    df_raw = pd.read_excel(file_path, sheet_name=sheet, header=None)

    # Suburb names (column 1, starting row 3)
    suburbs = df_raw.iloc[3:, 1].reset_index(drop=True)
    suburbs.name = "Suburb"

    # Date row and type row
    date_row = df_raw.iloc[1]
    type_row = df_raw.iloc[2]

    # Keep only "Median" columns
    median_cols = [i for i, t in enumerate(type_row) if t == "Median"]

    records = []
    for col in median_cols:
        date_str = str(date_row[col]).strip()
        try:
            date = pd.to_datetime(date_str, format="%b %Y")
        except:
            date = pd.NaT

        rents = df_raw.iloc[3:, col].reset_index(drop=True)
        for suburb, rent in zip(suburbs, rents):
            if pd.isna(suburb) or suburb == "Group Total":
                continue
            records.append((suburb, date, pd.to_numeric(rent, errors="coerce"), sheet))

    df_tidy = pd.DataFrame(records, columns=["Suburb", "date", "median_rent", "property_type"])
    all_data.append(df_tidy)

# Combine all sheets
tidy = pd.concat(all_data, ignore_index=True)
tidy = tidy.dropna(subset=["date", "median_rent"])

print("Final tidy shape:", tidy.shape)
display(tidy.head(20))

Sheets included: ['1 bedroom flat', '2 bedroom flat', '3 bedroom flat', '2 bedroom house', '3 bedroom house', '4 bedroom house']
Final tidy shape: (84729, 4)


,Suburb,date,median_rent,property_type
0,Albert Park-Middle Park-West St Kilda,2000-03-01,165.0,1 bedroom flat
1,Armadale,2000-03-01,150.0,1 bedroom flat
2,Carlton North,2000-03-01,150.0,1 bedroom flat
3,Carlton-Parkville,2000-03-01,165.0,1 bedroom flat
4,CBD-St Kilda Rd,2000-03-01,250.0,1 bedroom flat
5,Collingwood-Abbotsford,2000-03-01,135.0,1 bedroom flat
7,East Melbourne,2000-03-01,180.0,1 bedroom flat
8,East St Kilda,2000-03-01,140.0,1 bedroom flat
9,Elwood,2000-03-01,150.0,1 bedroom flat
10,Fitzroy,2000-03-01,140.0,1 bedroom flat
